# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [23]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [24]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [25]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []     
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)      
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
             
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [26]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


Part II.  Apache Cassandra coding. 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [27]:
#  connection to a Cassandra instance 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [28]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [29]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [30]:
# music_library table is created to retrieve data based on key partition sessionIda and itemInSession

query = "create table if not exists music_library"
query = query + "(sessionId int,itemInSession int,Artist_name text,Song_Title text, Song_length float,primary key(sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
    
#2. artist_library table is created to retrieve data based on key partition userid , sessionid.

    
query = "create table if not exists artist_library"
query = query + "(User_id int,sessionId int,itemInSession int,Artist_name text,Song_Title text, first_name text, last_name text,primary key((user_id, sessionid), iteminsession)); "
try:
    session.execute(query)
except Exception as e:
    print(e)    

    
# 3. Users table is created to retrieve data based on key partition song_title, user_id.

query = "create table if not exists Users"
query = query + "(Song_Title text,user_id int,first_name text, last_name text,primary key(song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [31]:
# inserting roes into music_library 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (sessionId,itemInSession,Artist_name,Song_Title, Song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [32]:
# Query to verify the data was entered into the table.
query = "select artist_name,Song_Title,Song_length from music_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [33]:
# inserting rows into artist_library. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (User_id ,sessionId ,itemInSession  ,Artist_name ,Song_Title , first_name , last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[3],line[4]))
                    

In [34]:
#inserting roes into artist_library

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (Song_Title,user_id,first_name,last_name)"
        query = query + "VALUES (%s,%s,%s, %s)"
        session.execute(query, (line[9],int(line[10]), line[4],line[5]))

                    

In [35]:
# Query to verify the data was entered into the table
query = "select artist_name,Song_Title,first_name,last_name from artist_library WHERE user_id = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist_name, row.song_title, row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On 0 Cruz
Three Drives Greece 2000 1 Cruz
Sebastien Tellier Kilometer 2 Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Cruz


In [36]:
# Query to verify the data was entered into the table
query = "select first_name,last_name from users WHERE Song_Title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.first_name,row.last_name)

Lynch 196.91057
Levine 196.91057
Johnson 196.91057


### Drop the tables before closing out the sessions

In [37]:
session.execute("drop table music_library ")
session.execute("drop table artist_library ")
session.execute("drop table Users")

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()